<font size = 6 color = 'blue'>
<center> Notebook 3: Clasificador de rocas utilizando Keras Tuner y Transfer learning </center>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install keras-tuner -q

     |████████████████████████████████| 97 kB 3.6 MB/s 


In [3]:
import time
import itertools
import numpy as np
import random as rn
import pandas as pd

import tensorflow as tf
from tensorflow import keras

#El importante aquí
import keras_tuner as kt

import matplotlib.pyplot as plt
from matplotlib.pyplot import imread

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as PRFS

from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

#from keras.preprocessing.image import ImageDataGenerator

rn.seed(1)
np.random.seed(1)

In [4]:
keras.mixed_precision.set_global_policy("mixed_float16")
#keras.mixed_precision.set_global_policy("mixed_float16")

Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  Tesla P100-PCIE-16GB, compute capability 6.0
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [5]:
!nvidia-smi

Thu Aug 26 11:13:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Datos

In [6]:
df = pd.read_pickle('/content/drive/MyDrive/Clasificador_from_scratch/images_for_training.pkl')

print("El dataframe con los datos para entrenar tiene {} arreglos correspondientes a las imágenes en la base aumentada."\
      .format(len(df)))

El dataframe con los datos para entrenar tiene 1815 arreglos correspondientes a las imágenes en la base aumentada.


In [7]:
imagenes = np.array( df["images"][:] )

In [8]:
images_accurate = []
images_with_issues = []
index = []

for i in range(len(imagenes)):
    if imagenes[i].shape != (224,224,3):
        
            image = imagenes[i]   
            images_with_issues.append(image)
            index.append(i)
    
        
    image = imagenes[i]   
    images_accurate.append(image)
    
print("Los índices de las imágenes con errores son: {}. \nEn total {} imágenes a eliminar."\
      .format(index, len(images_with_issues)))

Los índices de las imágenes con errores son: [388, 464, 593, 935, 939, 1132, 1378, 1682, 1703]. 
En total 9 imágenes a eliminar.


In [9]:
for i in range(len(index)):
    df = df.drop(index[i])
    
print("El nuevo tamaño del Dataframe es {}".format(len(df)))

El nuevo tamaño del Dataframe es 1806


In [10]:
images = np.array(df["images"][:])

X = []

for i in range(len(images)):
    
    image = images[i]   
    
    X.append(image)
    
print("El tamaño de la lista X la cual contiene las variables de entrada es {}".format(len(X)))

El tamaño de la lista X la cual contiene las variables de entrada es 1806


In [11]:
X_norm = np.array(X) / 255

In [12]:
y = np.array(df["category"][:])
print("La dimensión de y era {}".format(y.shape))

y = y.reshape((y.shape[0],1))
print("La dimensión de y ahora es {}".format(y.shape))

La dimensión de y era (1806,)
La dimensión de y ahora es (1806, 1)


In [13]:
y_one_hot = to_categorical(y)

In [14]:
idx = np.random.permutation(len(X_norm))

X_norm_permut, y_one_hot_permut = X_norm[idx], y_one_hot[idx]

In [15]:
# Calculamos el número de arreglos o imágenes correspondientes al conjunto de validación
val_split_num = int(round( 0.10 * len(y) ))
print("Hay un total de {} imágenes para validar".format(val_split_num))

# Calculamos el número de arreglos o imágenes correspondientes al conjunto de prueba
test_split_num = int(round( 0.10 * len(y) ))
print("\nHay un total de {} imágenes para el conjunto de prueba".format(test_split_num))

Hay un total de 181 imágenes para validar

Hay un total de 181 imágenes para el conjunto de prueba


In [16]:
train_x = X_norm_permut[test_split_num + val_split_num:]
train_y = y_one_hot_permut[test_split_num + val_split_num:]

val_x = X_norm_permut[test_split_num: test_split_num + val_split_num]
val_y = y_one_hot_permut[test_split_num: test_split_num + val_split_num]

test_x = X_norm_permut[:test_split_num]
test_y = y_one_hot_permut[:test_split_num]

In [17]:
print ("train_x shape: " + str(train_x.shape))
print ("train_y shape: " + str(train_y.shape))

print ("val_x shape: " + str(val_x.shape))
print ("val_y shape: " + str(val_y.shape))

print ("test_x shape: " + str(test_x.shape))
print ("test_y shape: " + str(test_y.shape))

train_x shape: (1444, 224, 224, 3)
train_y shape: (1444, 7)
val_x shape: (181, 224, 224, 3)
val_y shape: (181, 7)
test_x shape: (181, 224, 224, 3)
test_y shape: (181, 7)


# Modelo Keras-Tuner

In [20]:
conv_base = tf.keras.applications.mobilenet.MobileNet(input_shape=(224,224,3), alpha=1.0, 
                                                      depth_multiplier=1, 
                                                      dropout=0.001,
                                                      include_top=False,
                                                      pooling="max",
                                                      weights='imagenet'
                                                      )

17235968/17225924 [==============================] - 0s 0us/step


In [21]:
#Definimos las transformaciones a las imágenes en la Data Augmentation

data_augmentation = keras.Sequential(
[
tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
]
)

In [26]:
def build_model_conv(hp):
  
  inputs = keras.Input(shape=(224, 224, 3)) ## CAMBIAR EN CASO DE UNA DISTRIBUCIÓN DE DATOS DIFERENTE.
  
  x = inputs


  #Bloque Data Augmentation
  for i in range(hp.Int(name = 'DA_blocks', min_value=1, max_value=8, step=1)):
    
    x = data_augmentation(x)
    
    
  x = conv_base(x)

  x = tf.keras.layers.Flatten()(x)
  
  for i in range(hp.Int(name = 'Dense_blocks', min_value=1, max_value=3, step=1)):

    nodes = hp.Int(name = 'Nodes', min_value=128, max_value=384, step=64)
    x = Dense(nodes, activation='relu')(x)
  

  x = BatchNormalization()(x)

  x = Dropout(0.5)(x)
  
  outputs = Dense(7, activation="softmax")(x)

  model = tf.keras.Model(inputs, outputs)

  model.compile(optimizer = "adam" , loss =  "categorical_crossentropy", metrics = "accuracy")

  return model

In [27]:
#Definimos la métrica objetivo a maximizar en este caso
objective = kt.Objective( name="val_accuracy",
                         direction="max")

In [28]:
# Definción del Keras Tuner como "Bayesian Optimization" un optimizador que a partir de los resultados de los entrenamientos pasados genera
# nuevos valores de hiperparámetros.

tuner_conv = kt.BayesianOptimization(build_model_conv,
                                #Buscamos maximizar la métrica accuracy en el conjunto de validación
                                objective=objective,
                                #Número de entrenamientos con diferentes hparámetros máximo
                                max_trials=50,
                                #Número de veces que se entrenará el mismo modelo buscando mejorar el valor de la métrica 
                                executions_per_trial=2,
                                #Ruta donde almancenaron los registros de una búsqueda previemente iniciada y se cargan para continuar con la búsqueda
                                #O ruta de donde se almacenarán los registros de búsqueda de un entrenamiento nuevo
                                #La elección de una de las anteriores depende de lo especificado en "overwrite".
                                directory="/content/drive/MyDrive/Clasificador_from_scratch/PruebasTuner_2/",
                                #Si "overwrite" se encuentra en "False", se cargan los datos de búsquedas de hps pasadas y se retoma desde ahí.
                                #Si "overwrite" se encuentra en "True", se borra el historial previo y se genera una nueva búsqueda
                                overwrite=True
                                )

<font size = 4 color='black'>

Sobre la <i>__regresión lineal bayesiana__</i>. Más información [aquí](https://www.cs.toronto.edu/~rgrosse/courses/csc411_f18/slides/lec19-slides.pdf).


In [29]:
tuner_conv.search_space_summary()

Search space summary
Default search space size: 3
DA_blocks (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 8, 'step': 1, 'sampling': None}
Dense_blocks (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
Nodes (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 384, 'step': 64, 'sampling': None}


In [30]:
#Definimos los callbacks

callbacks = [keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=5)#,
             #keras.callbacks.ReduceLROnPlateau(monitor='val_mean_absolute_error', mode='min', patience=3, factor=0.1, verbose=1)
             #keras.callbacks.TensorBoard(log_dir="logs/model", histogram_freq=1)
             ]

In [31]:
#El método ".search()" permite es análogo al método ".fit()"
#Entrenamos

tuner_conv.search(train_x, train_y,
                  batch_size=128,
                  epochs=100,
                  validation_data=(val_x, val_y),
                  callbacks=callbacks,
                  verbose=1
                  )

Trial 50 Complete [00h 01m 27s]
val_accuracy: 0.9502762258052826

Best val_accuracy So Far: 0.9585635364055634
Total elapsed time: 01h 51m 50s
INFO:tensorflow:Oracle triggered exit


In [32]:
#Una vez obtenidos los conjuntos de hiperparámetros con mejor desempeño seleccionamos el top n:

#Definir un top
top_n = 3
#Optenemos los mejores hps
best_hps = tuner_conv.get_best_hyperparameters(top_n)

In [33]:
#Una vez optenidos los mejores conjuntos de hps.
#Definimos la función "get_best_epoch(hp)" que para cada conjunto de hiperparámetros (hp), definirá una arquitectura con la función 
#"build_model_conv(hp)", entrenará en el mismo conjunto de train y val y utilizando un callback EarlyStopping() con un valor de paciencia agresivo.
#La idea de tener un valor agresivo en la paciencia es entrenar cada uno de estos modelos (arquitectura + optimizador + datos) lo mejor posible y esto implica
#entrenar el mayor tiempo posible (en datos y tiempo de entrenamiento, más siempre es mejor).

#Una vez que el modelo termina su entrenamieno, el valor que regresa la función "get_best_epoch(hp)" es la última época que entrenada hasta donde el 
#EarlyStopping() detuvo el entrenamiento y que tuvo el valor más bajo en la métrica elegida.

#El objetivo de obtener la mejor época (aquella donde la métrica elegida ya decrecía o se incrementaba) es ahorrar tiempo de entrenamiento.


def get_best_epoch(hp):
  
  #Definimos la arquitectura con la función "build_model(hp)"
  model = build_model_conv(hp)
  
  #Definición de los callbacks
  callbacks=[keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=10)]

  #Entrenamiento
  history = model.fit(
      train_x, train_y,
      validation_data=(val_x, val_y),
      epochs=100,
      batch_size=128,
      callbacks=callbacks)
  
  val_loss_per_epoch = history.history["val_loss"]
  
  #Selección mejor época (valor mínimo en la métrica)
  best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
  
  print(f"Best epoch: {best_epoch}")
  return best_epoch

In [36]:
#Una vez que se conoce la mejor época se entrena el modelo pero con el total de los datos.
#Ya no existe una necesidad de estar monitoreando el error y la exactitud en el conjunto de validación, así que en este entrenamiento es una buena
#práctica entrenar sobre todos los datos.

def get_best_trained_model(hp):
  
  #Obtenemos la mejor época
  best_epoch = get_best_epoch(hp)
  
  #Con el mismo conjunto de hp construimos el modelo
  model = build_model_conv(hp)

  #Definimos callbacks de los mejores modelos
  callbacks = [keras.callbacks.EarlyStopping(monitor="loss", mode="min", patience=10)
              ]

  #Entrenamos el modelo con los datos totales y una cantidad de épocas apenas superior a la mejor época.
  history = model.fit(train_x, train_y,
            batch_size=128,
            epochs=int(best_epoch * 1.2),
            callbacks = callbacks)
  

  return model

In [37]:
#Las funciones arriba definidas ahora entran en juego.

"""
La idea del siguiente "for" loop es la siguiente, para cada conjunto de hiperparámetros (hp) seleccionado en la sección "Selección de los mejores modelos":
  
  1.- Obtener la mejor época. Con todo lo que eso implica.

  2.- Entrenar el modelo definido por ese conjunto de hps en el total de los datos y por una cantidad de épocas apenas mayor a la mejor.

  3.- Evaluar el desempeño de ese modelo en el conjunto de test.

"""

#Lista donde se almacenarán los modelos definidos por la lista "best_hps"
best_models = []

for hp in best_hps:

  #Obtención del modelo ya evaluado en el total de los datos (train + val).
  #OBS: recordar que primero se tiene que optener la mejor época.
  model = get_best_trained_model(hp)
  
  print("Evaluando el modelo...\n")
  print("El modelo obtuvo los resultados siguientes en el conjunto TEST:")
  print(model.evaluate(test_x, test_y))
  print("\n")
  
  #Agregamos el modelo a la lista "best_models()"
  best_models.append(model)

Epoch 1/100
12/12 [==============================] - 11s 395ms/step - loss: 0.3636 - accuracy: 0.9120 - val_loss: 3.1705 - val_accuracy: 0.8122
Epoch 2/100
12/12 [==============================] - 4s 327ms/step - loss: 0.0443 - accuracy: 0.9861 - val_loss: 1.8639 - val_accuracy: 0.8398
Epoch 3/100
12/12 [==============================] - 4s 327ms/step - loss: 0.0241 - accuracy: 0.9952 - val_loss: 1.9297 - val_accuracy: 0.8287
Epoch 4/100
12/12 [==============================] - 4s 327ms/step - loss: 0.0280 - accuracy: 0.9917 - val_loss: 1.6997 - val_accuracy: 0.8398
Epoch 5/100
12/12 [==============================] - 4s 329ms/step - loss: 0.0269 - accuracy: 0.9924 - val_loss: 1.0529 - val_accuracy: 0.8729
Epoch 6/100
12/12 [==============================] - 4s 327ms/step - loss: 0.0169 - accuracy: 0.9938 - val_loss: 1.0457 - val_accuracy: 0.8840
Epoch 7/100
12/12 [==============================] - 4s 328ms/step - loss: 0.0153 - accuracy: 0.9952 - val_loss: 0.9379 - val_accuracy: 0.895

In [38]:
#Una vez definidos los mejores modelos podemos guardarlos como un archivo ".h5" ejecutando la siguiente celda

path_to_save_models = "/content/drive/MyDrive/Clasificador_from_scratch/Mejores_modelos_tuner/"  #DEFINIR RUTA DE DÓNDE GUARDAR LOS MEJORES MODELOS

for model in best_models:
  model.save(path_to_save_models + str(model),
             overwrite=True,
             include_optimizer=True,
             save_format="h5")

### ¿Dónde revisar con más detalle Keras Tuner?

[Aquí](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html) y [aquí](https://www.manning.com/books/deep-learning-with-python-second-edition?query=Deep%20Learning%20with%20Python%202).

¡Gracias!